In [482]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#dts
M1=np.zeros((19,106)) #进口：对应1
M3=np.zeros((19,106)) #国产：对应2
#ctd
M2=np.zeros((1,18)) #进口
M4=np.zeros((1,18)) #国产

#仓库状态矩阵
O=np.zeros((20,1))  #是否开仓
Z=np.zeros((20,1))  #是否智能


In [483]:
#参数导入
#国产，对应2
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [484]:
#港口，对应1
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))
Q_dm.shape

(1, 106)

In [485]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,11,axis=0)  #im(delete neilu)
#k = 10 
#row_to_move = L_dts_1[k]
#L_dts_1 = np.delete(L_dts_1, k, axis=0)
#L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,6,axis=0)  #dm(delete gangkou)
#k = 6 
#row_to_move = L_dts_2[k]
#L_dts_2 = np.delete(L_dts_2, k, axis=0)
#L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2.shape

(19, 106)

In [486]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_2=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_1.shape

(19, 106)

In [487]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]

In [488]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

In [489]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1:]
Pf.shape

(20, 2)

In [490]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
#Of.reshape((1,20))
Of.shape

(20,)

In [491]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
#Cap.reshape((1,20))
Cap.shape

(20,)

In [492]:
#约束条件
##时间
Q_all=Q_im.sum()
def time_point(M_im,M_dm):
    Q_out=0
    tmp1=M_im*T_dts_1
    tmp2=M_dm*T_dts_2  #这里改到对应的时间矩阵
    #print(tmp1,"\n",tmp2)
    tmp1=tmp1.sum(axis=0)
    tmp2=tmp2.sum(axis=0)
    for i in range(0,106):
        if tmp1[i]>=10:
            Q_out+=Q_im[0,i]
        if tmp2[i]>=10:
            Q_out+=Q_dm[0,i]
    out_point=Q_out/Q_all
    return out_point

def time_lim(M_im,M_dm):
    if time_point(M_im,M_dm)<=0.05:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim(M_im,M_dm):
    tmp=M_im*Q_im
    tmp1=tmp.sum(axis=1)
    tmp2=M_dm*Q_dm
    tmp3=tmp2.sum(axis=1)
    s1=0
    s2=0
    for j in range(0,20):
        if j==6:
            sum=tmp1[s1]
            s2-=1
        elif j==11:
            sum=tmp3[s2]
            s1-=1
        else:
            sum=tmp1[s1]+tmp3[s2]
        if sum>Cap[j]:
            return False
        s1+=1
        s2+=1
    return True

#cdc必须开
def cdc_op(O):
    return O[6]==1 and O[11]==1

#每个门店对应一个商店
def store(M_1,M_3):
    M1_s=M_1.sum(axis=0)
    #print(M1_s.shape)
    M2_s=M_3.sum(axis=0)
    #print(M2_s)
    for i in M1_s:
        if i>1:
            return False
    for i in M2_s:
        if i>1:
            return False
    return True


M1t=np.random.randint(0,2,size=(19,106))
M3t=np.random.randint(0,2,size=(19,106))
print(cap_lim(M1t,M3t))

False


In [493]:
def cp(O,Z,Q_dm,Q_im,M_dm,M_im,P):
    cp = np.zeros((1,19))
    Q_c_dm = np.zeros((19,1))
    Q_c_im = np.zeros((19,1))
    Q_c_dm = M_dm @ Q_dm.T
    Q_c_im = M_im @ Q_im.T
    Q_c = Q_c_dm[1:] + Q_c_im[1:]
    Q_c = np.insert(Q_c, Q_c_dm[0],6)
    Q_c = np.insert(Q_c, Q_c_im[0],11)
    for i in range(0,20):
        fl=Z[i]
        cp[i]=fl+O[i]+Q_c[i]*P[i,fl]
    return np.sum(cp), Q_c


In [494]:
#目标优化式
##处置成本
def ccl_cp(O,Z,Q,M_s,M_d,P,k):
    cp=0
    tmp=M_s*Q  
    tmp1=tmp.sum(axis=1)
    #print(tmp1)
    tmp1=tmp1.reshape((1,19))
    tmp11=np.delete(tmp1,k,axis=1)
    tmp2=M_d*tmp11
    ccp=tmp2.sum()
    for i in range(0,19):
        fl=Z[i]
        cp+=fl+O[i]+tmp1.T[i]*P[i][fl]
    return cp+ccp
#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    #ct1=np.zeros((1,20))
    sum=0
    tmp1 = M_s * Q * L_s * 1.25
    Q_c = Q @ M_s.T
    Q_c = np.delete(Q_c,k,axis=1) 
    tmp2= M_d * Q_c * L_d.T * 0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z):
    c=0
    #进口
    c+=ccl_cp(O,Z,Q_im,M1,M2,Pf,6)
    print(c)
    c+=ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    print(c)
    #国产
    c+=ccl_cp(O,Z,Q_dm,M3,M4,Pf,10)
    print(c)
    c+=ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    print(c)
    #c += cp(O,Z,Q_dm,Q_im,M1,M3,Pf)

    #c += ccl_ct(Q_im,M1,M2,L_dts_1,L_ctd[0]) + ccl_ct(Q_dm,M3,M4,L_dts_2,L_ctd[1])
    return c

In [495]:
def test():
    M1t=np.random.randint(0,2,size=(19,106))
    M2t=np.random.randint(0,2,size=(1,18))
    M3t=np.random.randint(0,2,size=(19,106))
    M4t=np.random.randint(0,2,size=(1,18))
    Ot=np.random.randint(0,2,size=(20,1))
    Zt=np.random.randint(0,2,size=(20,1))
    a=cal_all(M1t,M2t,M3t,M4t,Ot,Zt)

test()


[15883.003825]
[74815012.38382499]
[74842116.223375]
[1.86648191e+08]


In [496]:
from gurobipy import *
m=Model("ass_mov")
m.addVars(M1,M2,M3,M4,O,Z,vtype=GRB.BINARY)
m.update()
m.setObjective(cal_all(M1,M2,M3,M4,O,Z),GRB.MINIMIZE)
m.addConstrs(time_lim(M1,M3,Q_im,Q_dm))
m.addConstrs(cap_lim(M1,M3))
m.addConstrs(cdc_op(O))
m.optimize()
print("M1:",M1,"M2",M2)

TypeError: unhashable type: 'numpy.ndarray'